In [3]:
import numpy as np
import pandas as pd
import psycopg2
import sqlalchemy
import matplotlib.pyplot as plt

In [4]:
import config
from sqlalchemy import create_engine

cnx = create_engine(config.postgres_str)

In [5]:
head = pd.read_sql_query('''SELECT * FROM hpd_violations LIMIT 100;''', cnx)
head.columns

Index(['violationid', 'buildingid', 'registrationid', 'boroid', 'borough',
       'housenumber', 'lowhousenumber', 'highhousenumber', 'streetname',
       'streetcode', 'postcode', 'apartment', 'story', 'block', 'lot', 'class',
       'inspectiondate', 'approveddate', 'originalcertifybydate',
       'originalcorrectbydate', 'newcertifybydate', 'newcorrectbydate',
       'certifieddate', 'ordernumber', 'novid', 'novdescription',
       'novissueddate', 'currentstatusid', 'currentstatus',
       'currentstatusdate', 'novtype', 'violationstatus', 'latitude',
       'longitude', 'communityboard', 'councildistrict', 'censustract', 'bin',
       'bbl', 'nta'],
      dtype='object')

In [6]:
top_violators = pd.read_sql_query('''SELECT count(*), buildingid FROM hpd_violations GROUP BY buildingid HAVING COUNT(*) > 1 LIMIT 500;''', cnx)
top_100 = top_violators.sort_values(by=['count'], ascending = False).head(100)
top_100

,count,buildingid
317,1010,5378
217,772,81857
368,577,92616
380,480,491277
192,452,40954
112,437,64090
131,386,337901
440,368,59926
401,361,43016
373,281,21312


# use bbl to visualize on a map

In [11]:
top100_buildingids = top_100['buildingid']
top100_buildingids.head()

317      5378
217     81857
368     92616
380    491277
192     40954
Name: buildingid, dtype: int64

In [16]:
# df with all the violations on top 100 buildingid
top100_df = pd.read_sql_query(f'''SELECT * FROM hpd_violations WHERE buildingid = {top100_buildingids[0]}''', cnx)

In [19]:
for i in range(1, len(top100_buildingids)):
    temp = pd.read_sql_query(f'''SELECT * FROM hpd_violations WHERE buildingid = {top100_buildingids.iloc[i]}''', cnx)
    top100_df = pd.concat([top100_df, temp])
top100_df['buildingid'].unique()

array([909820,  81857,  92616, 491277,  40954,  64090, 337901,  59926,
        43016,  21312, 428409, 209483, 309621,  28015, 187420, 115231,
       292856, 280157, 348545, 812433,  41887, 112833, 343413, 808241,
       362833, 161569,  73800,  56824, 332466, 112516,  73719,  40971,
       186195,  21273,  45433,  66093,   5682, 373078, 295966,  44088,
       371724, 309139, 408778,  62215, 299120, 427765, 296558, 733699,
       363406,  19130, 509336,  73672, 373513, 379928, 460495,  27721,
       359803,  39256, 398829, 240473,  56255, 121331, 818167, 461388,
       312123,  39154, 957363, 339313, 268499, 203922,  28365, 351644,
       217582, 213576, 675171, 890315,  59315, 217258,  31276,  58985,
       660468, 417170,  96295, 114196, 305979, 349528, 347770, 559270,
       357648, 179627,  82081, 343596, 117113, 208106,  36049, 527559,
       347464, 334145, 222393], dtype=int64)

In [25]:
top100_bbl = top100_df['bbl'].unique()